## Semi-auto Stream in Landsat from USGS
Solution: Use brower-cookie3 to "sign" the URL request so a python script can stream data from USGS
Cons: Low bandwidth ~200KB/s 

In [ ]:
#%pip install browser-cookie3

In [102]:
# background step: login in USGS EROS so the brower cookie can skip the redirect of USGS when request
import browser_cookie3
cj = browser_cookie3.chrome()

In [103]:
cj

<CookieJar[Cookie(version=0, name='uid', value='960569e0-5d65-11ec-ae6b-901b0ea4a41b', port=None, port_specified=False, domain='.1dmp.io', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=1, expires=1671081143, discard=False, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='_rxuuid', value='%7B%22rx_uuid%22%3A%22RX-da54f3a6-14be-4bb6-9223-fa36c2e0337c-005%22%2C%22nxtrdr%22%3Afalse%7D', port=None, port_specified=False, domain='.1rx.io', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=1, expires=1671123274, discard=False, comment=None, comment_url=None, rest={'HTTPOnly': ''}, rfc2109=False), Cookie(version=0, name='33x_ps', value='u%3D118839105538212%3As1%3D1639586775495%3Ats%3D1639586775495', port=None, port_specified=False, domain='.33across.com', domain_specified=True, domain_initial_dot=True, path='/', path_specified=True, secure=1, expires=1671122858, discard=False, comment=Non

In [47]:
url = "https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2020/047/027/LC08_L2SP_047027_20201204_20210313_02_T1/LC08_L2SP_047027_20201204_20210313_02_T1_SR_B7.TIFa"
url = "https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2022/008/009/LC09_L1GT_008009_20221014_20221014_02_T2/LC09_L1GT_008009_20221014_20221014_02_T2_B2.TIF"

In [49]:
from pathlib import Path
fname = Path(url).name

In [99]:
url

'https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_MTL.xml'

In [100]:
import requests
r = requests.get(url, stream = True,cookies=cj)

In [101]:
import shutil
fname = Path(url).name
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
    with open(fname,'wb') as f:
        shutil.copyfileobj(r.raw, f)

In [79]:
def cache_asset(url,path):
    r = requests.get(url, stream = True,cookies=cj)
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True
        
        # Open a local file with wb ( write binary ) permission.
        with open(path,'wb') as f:
            shutil.copyfileobj(r.raw, f)

In [54]:
from pystac.item import Item

In [ ]:
usgs_item_url = "https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l1/items/LC09_L1TP_094083_20211110_20220119_02_T1"
aws_item_url = "https://jcz3phgts3.execute-api.us-west-2.amazonaws.com/dev/getobject/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_stac.json"

In [57]:
scene = Item.from_file("https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l1/items/LC09_L1TP_094083_20211110_20220119_02_T1")

In [78]:
scene.id

'LC09_L1TP_094083_20211110_20220119_02_T1'

In [91]:
assets_urls = [[k,scene.assets[k].href] for k in scene.get_assets().keys() if k != 'index']

In [96]:
import os
directory = f"../../data/landsat/hand_pull/{scene.id}"
if not os.path.exists(directory):
    os.makedirs(directory)
    


In [98]:
for k,url in assets_urls:
    
    
    path = f"{directory}/{Path(url).name}"
    print(k,url,path)
    cache_asset(url,path)



thumbnail https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_thumb_small.jpeg ../../data/landsat/hand_pull/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_thumb_small.jpeg
reduced_resolution_browse https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_thumb_large.jpeg ../../data/landsat/hand_pull/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_thumb_large.jpeg
MTL.json https://landsatlook.usgs.gov/data/collection02/level-1/standard/oli-tirs/2021/094/083/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_MTL.json ../../data/landsat/hand_pull/LC09_L1TP_094083_20211110_20220119_02_T1/LC09_L1TP_094083_20211110_20220119_02_T1_MTL.json
coastal https://landsatlook.usgs

In [ ]:
path = Path()

In [1]:
import pandas as pd
from dateutil.rrule import rrule, DAILY
import urllib.request, json 

## collect inventory from USGS
Query guide: https://landsatlook.usgs.gov/stac-server/api.html#tag/Item-Search


Example URL:
https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l1/items?limit=10000&datetime=2022-01-06T00:00:00Z/2022-01-07T00:00:00Z&fields=id,-type,-geometry,-bbox,-properties,-links,-assets,-collection,-features

In [3]:
from datetime import date

a = date(2022, 9, 30)
b = date(2022, 10, 8)

for dt in rrule(DAILY, dtstart=a, until=b):
    print(dt.strftime("%Y-%m-%d"))
    yymmdd = dt.strftime("%Y-%m-%d")
    dt

2022-09-30
2022-10-01
2022-10-02
2022-10-03
2022-10-04
2022-10-05
2022-10-06
2022-10-07
2022-10-08


In [19]:
cltns = ["landsat-c2l2-sr",	# Landsat Collection 2 Level-2 UTM Surface Reflectance (SR) Product
"landsat-c2l2-st",	# Landsat Collection 2 Level-2 UTM Surface Temperature (ST) Product
"landsat-c2ard-st",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Temperature (ST) Product
"landsat-c2l2alb-bt",	# Landsat Collection 2 Level-2 Albers Top of Atmosphere Brightness Temperature (BT) Product
"landsat-c2l3-fsca",	# Landsat Collection 2 Level-3 Fractional Snow Covered Area (fSCA) Product
"landsat-c2ard-bt",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere Brightness Temperature (BT) Product
"landsat-c2l1",	# Landsat Collection 2 Level-1 Product
"landsat-c2l3-ba",	# Landsat Collection 2 Level-3 Burned Area (BA) Product
"landsat-c2l2alb-st",	# Landsat Collection 2 Level-2 Albers Surface Temperature (ST) Product
"landsat-c2ard-sr",	# Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Surface Reflectance (SR) Product
"landsat-c2l2alb-sr",	# Landsat Collection 2 Level-2 Albers Surface Reflectance (SR) Product
"landsat-c2l2alb-ta",	# Landsat Collection 2 Level-2 Albers Top of Atmosphere (TA) Reflectance Product
"landsat-c2l3-dswe",	 #Landsat Collection 2 Level-3 Dynamic Surface Water Extent (DSWE) Product
"landsat-c2ard-ta"]    #	Landsat Collection 2 Analysis Ready Data (ARD) Level-2 UTM Top of Atmosphere (TA) Reflectance Product

In [20]:
import os


for cltn in cltns:
    directory = f"../../data/landsat/stac/{cltn}"
    if not os.path.exists(directory):
        os.makedirs(directory)


In [27]:
cltn = "landsat-c2l3-fsca"

In [28]:
url = f'''https://landsatlook.usgs.gov/stac-server/collections/{cltn}/items?limit=10000&datetime={yymmdd}T00:00:00Z/{yymmdd}T23:59:59Z&fields=id,-type,-geometry,-bbox,properties,-links,-assets,-collection,-features'''

In [29]:
stat_list = []

In [34]:

    print('has')

has


In [30]:

with urllib.request.urlopen(url) as uo:
    data = json.load(uo)
    #print(data)
    if not data['features']:
    feature_df = pd.json_normalize(data['features'])
    l9 = feature_df.query('`properties.platform`=="LANDSAT_9"')
    l8 = feature_df.query('`properties.platform`=="LANDSAT_8"')
    stat_list.append({'date':yymmdd,'total':data['numberMatched'],'return':data['numberReturned'],'LC09':len(l9),'LC08':len(l9)})
    feature_df.drop('type',axis=1).to_csv(f'../../data/landsat/stac/{cltn}_{dt.strftime("%Y-%m-%d")}.csv',index=False)


UndefinedVariableError: name 'BACKTICK_QUOTED_STRING_properties_DOT_platform' is not defined

In [ ]:
pd.DataFrame(stat_list).query('`date` > "2021-09-27"').to_csv('../../data/landsat/day_sum_LC09_LC08.csv',index=False)

In [ ]:
day_sum = pd.DataFrame(stat_list).query('`date` > "2021-09-27"')

In [ ]:
pd.json_normalize(data['features']).query('`properties.platform`=="LANDSAT_9"')

### Batch processing

In [17]:
cltn

'landsat-c2l2-sr'

In [35]:
def inventory_collection(cltn):
    a = date(2021, 10, 15)
    b = date(2022, 10, 15)
    stat_list=[]
    for dt in rrule(DAILY, dtstart=a, until=b):
        #print(dt.strftime("%Y-%m-%d"))
        yymmdd = dt.strftime("%Y-%m-%d")
        url = f'''https://landsatlook.usgs.gov/stac-server/collections/{cltn}/items?limit=10000&datetime={yymmdd}T00:00:00Z/{yymmdd}T23:59:59Z&fields=id,-type,-geometry,-bbox,properties,-links,-assets,-collection,-features'''
        try:
            with urllib.request.urlopen(url) as uo:
                data = json.load(uo)
                #print(data)
                if not data['features']:
                    continue
                feature_df = pd.json_normalize(data['features'])
                l9 = feature_df.query('`properties.platform`=="LANDSAT_9"')
                l8 = feature_df.query('`properties.platform`=="LANDSAT_8"')
                stat_list.append({'date':yymmdd,'total':data['numberMatched'],'return':data['numberReturned'],'LC09':len(l9),'LC08':len(l8)})
                if len(l9)>0:
                    l9.drop('type',axis=1).to_csv(f'../../data/landsat/stac/{cltn}/{cltn}_{dt.strftime("%Y-%m-%d")}_L9.csv',index=False)
                #print(stat_list[-1])
        except e:
            print(e)
    pd.DataFrame(stat_list).to_csv(f'../../data/landsat/{cltn}_day_sum_LC09_LC08.csv',index=False)


In [36]:
for cltn in cltns[4:-1]:
    print(cltn)
    inventory_collection(cltn)

landsat-c2l3-fsca
landsat-c2ard-bt
landsat-c2l1
landsat-c2l3-ba
landsat-c2l2alb-st
landsat-c2ard-sr
landsat-c2l2alb-sr
landsat-c2l2alb-ta
landsat-c2l3-dswe


In [41]:
for cltn in cltns:
    print(cltn,pd.read_csv(f'../../data/landsat/{cltn}_day_sum_LC09_LC08.csv').LC09.sum())

landsat-c2l2-sr 207126
landsat-c2l2-st 175629
landsat-c2ard-st 40887
landsat-c2l2alb-bt 13857
landsat-c2l3-fsca 0
landsat-c2ard-bt 41984
landsat-c2l1 239920
landsat-c2l3-ba 0
landsat-c2l2alb-st 13611
landsat-c2ard-sr 41984
landsat-c2l2alb-sr 13857
landsat-c2l2alb-ta 13857
landsat-c2l3-dswe 0
landsat-c2ard-ta 41984


## compare file entries on local node
First run `find . -type f -name '*.tar' -exec basename {} .tar \; > landsat_avail.txt` on local inventory

In [ ]:
glad21 = pd.read_csv("../../../data/Landsat/landsat9_avail_2021.txt",names=['scene_id'], header=None)

In [ ]:
glad22 = pd.read_csv("../../../data/Landsat/landsat_avail_2022.txt",names=['scene_id'], header=None)

In [ ]:
glad_inventory = pd.concat([glad21,glad22[glad22.scene_id.str.startswith("LC09")]])

In [ ]:
import glob

In [ ]:
usgs_inventory = pd.concat([pd.read_csv(f) for f in glob.glob('../../../data/Landsat/stac/*.csv')])

In [ ]:
glad_inventory

In [ ]:
inventory_merged = usgs_inventory.merge(glad_inventory,left_on='id',right_on='scene_id',how='left')

In [ ]:
missing = inventory_merged.query('scene_id.isnull()',engine='python')

In [ ]:
missing.shape

In [ ]:
missing[['id']].to_csv('../../../data/Landsat/missing_id.csv',index=False)

In [ ]:
missing.dtypes

### Spatial and temporal visualization on the missing scenes

In [ ]:
import seaborn as sns

In [ ]:
pathrow_missing = missing.groupby(["properties.landsat:wrs_path","properties.landsat:wrs_row"]).agg({'id':'count'}).reset_index().set_axis(['path','row','count'],axis=1)\
    .pivot(index='row',columns='path',values='count').fillna(0)

In [ ]:
sns.heatmap(pathrow_missing,cbar_kws={'label': 'missing scenes'})


In [ ]:
missing['properties.datetime'] = missing['properties.datetime'].astype('datetime64[ns]')

In [ ]:
sns.displot(data=missing,x='properties.datetime',aspect=4,bins=360)